## CelFiE Quick Start 

**By Nick Semenkovich \<semenko@alum.mit.edu\>**

The original CelFiE project is by Christa Caggiano -- this simply reworks her code to be more practical and accept standard (.bed) inputs.

In this example, we download and run CelFiE-simplified, to predict one sample's fractional tissue abundance -- a single mature neutrophil from Blueprint.

Note: Note, CelFiE's TIM matrix was trained on Blueprint, so this is very overfit.

In [ ]:
# Install helpers
! apt-get -qq install bedops bedtools
! wget --no-config -q --no-clobber -O bigWigToWig https://hgdownload.cse.ucsc.edu/admin/exe/linux.x86_64.v385/bigWigToWig
! chmod +x bigWigToWig

In [ ]:
# Download CelFiE
! git clone https://github.com/semenko/celfie-simplified.git
! git -C celfie-simplified pull
print("Celfie git version: ", end = "")
! git -C celfie-simplified rev-parse --short HEAD

In [ ]:
# Download S00K5EA1, a mature neutrophil
# For details, see: http://dcc.blueprint-epigenome.eu/#/files

# bs_cov = total read counts
! wget --no-config -q --no-clobber -O S00K5EA1.bs_cov.bw http://ftp.ebi.ac.uk/pub/databases/blueprint/data/homo_sapiens/GRCh38/venous_blood/PB100713/mature_neutrophil/Bisulfite-Seq/CNAG/S00K5EA1.CPG_methylation_calls.bs_cov.GRCh38.20160531.bw

# bs_call = methylation percentage (not #)
! wget --no-config -q --no-clobber -O S00K5EA1.bs_call.bw http://ftp.ebi.ac.uk/pub/databases/blueprint/data/homo_sapiens/GRCh38/venous_blood/PB100713/mature_neutrophil/Bisulfite-Seq/CNAG/S00K5EA1.CPG_methylation_calls.bs_call.GRCh38.20160531.bw

In [ ]:
# Convert the two .bw files above into .beds
! ./bigWigToWig S00K5EA1.bs_cov.bw stdout | wig2bed > S00K5EA1.bs_cov.bed
! ./bigWigToWig S00K5EA1.bs_call.bw stdout | wig2bed > S00K5EA1.bs_call.bed

# Combine into one .bed, containing:
# chrom start stop %meth #depth
! paste S00K5EA1.bs_call.bed S00K5EA1.bs_cov.bed | cut -f 1-3,5,10 > S00K5EA1.percent_meth.bed

# Convert % methylation into counts, our required input format:
# chrom start stop #meth #depth
! awk 'BEGIN{OFS="\t"}{ print $1, $2, $3, int($4 * $5 + 0.5), int($5) }' S00K5EA1.percent_meth.bed > S00K5EA1.counts_meth.bed

In [ ]:
# Let's make sure that looks valid
! head -n 2 S00K5EA1.counts_meth.bed
! wc -l S00K5EA1.counts_meth.bed

# Optionally take a small sample for the git repo
# ! bedtools sample -n 100000 -seed 42 -i S00K5EA1.counts_meth.bed | sort -k 1,1 -k2,2n > sample-neutrophil.bed

In [ ]:
## Run CelFiE's sum_by_list -- basically intesects a .bed and sums two columns

# The TIM matrix can't have a header for this
! tail -n+2 celfie/tim_matrix.txt > celfie/tim_matrix.noheader.txt

# For speed, pre-filter the .bed to regions intersecting the TIM matrix
# -u prints only one entry of SAMPLE for each overlap with TIM_MATRIX
! bedtools intersect -u -a S00K5EA1.counts_meth.bed -b celfie/tim_matrix.noheader.txt > S00K5EA1.counts_meth.filtered.bed

! python celfie/TIMs/sum_by_list.py celfie/tim_matrix.noheader.txt S00K5EA1.counts_meth.filtered.bed S00K5EA1.summed.bed 1

In [ ]:
# Let's make sure that looks valid, too -- there should only be 1581 lines (one per TIM)
! head -n 2 S00K5EA1.summed.bed
! wc -l S00K5EA1.summed.bed

In [ ]:
# Now we need to stitch on a header
! echo -e "chrom\tstart\tend\tsample_meth\tsample_depth" > S00K5EA1.summed.with_header.txt
# We need to remove a ^M that sum_by_list.py leaves
! cat S00K5EA1.summed.bed | tr -d $'\r' >> S00K5EA1.summed.with_header.txt

# Combine with TIM matrix
! paste S00K5EA1.summed.with_header.txt celfie/tim_matrix.txt > S00K5EA1_celfie_input.txt

# Finally, does that look OK?
! head -n2 S00K5EA1_celfie_input.txt

In [ ]:
## Time to run CelFiE!
! python celfie-simplified/celfie.py -u 0 S00K5EA1_celfie_input.txt S00K5EA1_output 1

In [ ]:
# Let's plot it, using .ipynb code from CelFiE's demo
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
tissue_proportions = pd.read_csv("S00K5EA1_output/1_tissue_proportions.txt", delimiter="\t")
# Rename column 1
tissue_proportions.rename(columns={"Unnamed: 0": "samples"}, inplace=True)
# Melt to one entry per-line for sns plots
tissue_proportions = tissue_proportions.melt("samples",  var_name="tissue", value_name="estimate")
sns.boxplot(x="tissue", y="estimate", data=tissue_proportions, palette=["#61c2d3", "#003f4b"])
plt.title('S00K5EA1 Neutrophil')
plt.xticks(rotation=90)
plt.ylabel("CelFiE estimate")
plt.show()

In [ ]:
! cat S00K5EA1_output/1_tissue_proportions.txt